# Calcium Group Plotting & Statistics

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import scipy.io as sio
import lvm_read
from tkinter import Tk
from tkinter.filedialog import askdirectory
import pandas as pd
import peakutils
from collections import Counter
from scipy.signal import butter, lfilter, freqz
import plotly as py
import plotly.graph_objs as go
import glob


plt.style.use('seaborn-white')
%matplotlib inline

py.offline.init_notebook_mode(connected=True)

## Selecting the data folder
Should contain all .mat files

In [2]:
root = Tk()
ttl = "Select directory"

if os.name == 'nt':
    dir1 = 'C:\\'   # Windows
else:
    dir1 = '/Volumes/Zhivaz/'
    
root.withdraw()
file_path = askdirectory(initialdir = dir1, title = ttl)
root.update()
root.destroy()
print('main path :', file_path)

file_ending = "*.mat"
calcium_regressors = glob.glob(os.path.join(os.path.abspath(file_path),file_ending))
print('.mat files found: ',calcium_regressors)


main path : D:/MRIDATA/Test/newAnalysis/matfiles
.mat files found:  ['D:\\MRIDATA\\Test\\newAnalysis\\matfiles\\6756_E8.mat', 'D:\\MRIDATA\\Test\\newAnalysis\\matfiles\\6757_E9.mat']


In [3]:
alldata=[]
N_files=len(calcium_regressors)

for i in range(N_files):
    alldata.append(sio.loadmat(calcium_regressors[i]))
    
    
def get_nth_key(dictionary, n=0):
    if n < 0:
        n += len(dictionary)
    for i, key in enumerate(dictionary.keys()):
        if i == n:
            return key
    raise IndexError("dictionary index out of range") 

In [4]:
# N_files
# animallist = alldata[0].keys()
# alldata[0].values()
# animallist
# alldata[0]


# name_channel = get_nth_key(alldata[0],4)



#Create data frames
time_sec=alldata[0]['6756_E8 Channel 2']['time'][0][0][0]
index = pd.Index(time_sec)
columns = pd.MultiIndex.from_product([['channel1','channel2'],['mean','SD','stim']],names=['channel','datatype'])
# df0 = pd.DataFrame(np.zeros((index.size,columns.size)),index = index, columns = columns)

# df=[]
dfs=[]
measurements=[]
for i in range(N_files):
    measurementname = 'measurement ' + get_nth_key(alldata[i],4)[:-10]
    df = pd.DataFrame(np.zeros((index.size,columns.size)),index = index, columns = columns)
    
    df['channel1','mean'] = alldata[i][get_nth_key(alldata[i],3)]['mean'][0][0][0]
    df['channel1','SD'] = alldata[i][get_nth_key(alldata[i],3)]['SD'][0][0][0]
#     df['channel1','stim'] = alldata[0][get_nth_key(alldata[0],3)]['stim'][0][0][0]
    
    df['channel2','mean'] = alldata[i][get_nth_key(alldata[i],4)]['mean'][0][0][0]
    df['channel2','SD'] = alldata[i][get_nth_key(alldata[i],4)]['SD'][0][0][0]
#     df['channel2','stim'] = alldata[0][get_nth_key(alldata[0],4)]['stim'][0][0][0]
    measurements.append(measurementname)
    dfs.append(df)


dffinal=pd.concat(dfs, axis=1, names=['channel','datatype','scan'], ignore_index=False)

forplotting_mean = dffinal['channel1']['mean'].mean(axis=1)
forplotting_individual = dffinal['channel1']['mean']


In [5]:
shapes1 = list()
# for i in frange(0,block_duration,1/stim_freq):
#     shapes1.append({
#     'type': 'line',
#     'xref': 'x',
#     'yref': 'y',
#     'x0': i,
#     'y0': np.min(blocks.min(axis=1)),
#     'x1': i,
#     'y1': np.max(blocks.max(axis=1)),
#     'opacity':0.2,
#     'layer': 'above',
#     'line':{
#         'color':'red',
#         },
#     })

layout1 = go.Layout(
    title='responses',
    yaxis=dict(
    title='%'
    ),
    xaxis=dict(
    title='time [s]'
    )#,
#     shapes=shapes1
    )
        
traceA=[]
for i in range(forplotting_individual.columns.size):
    traceA.append(go.Scatter(x=forplotting_individual.index.values, y=forplotting_individual.iloc[:,i].values, mode = 'lines', name = 'individual response'))
traceA.append(go.Scatter(x=forplotting_mean.index.values, y=forplotting_mean.values, mode = 'lines', name = 'Mean Response'))
  
    
fig = go.Figure(data=traceA, layout=layout1)
py.offline.iplot(fig)   